## 10. Generating Predictions

In [1]:
/* Import dataset */
proc import 
    datafile='/home/u43674369/projects/project01/placement_data.csv' 
    out=df dbms=csv replace;
run;

/* Replace missing values with 0 */
proc stdize data=df
    out=df
    reponly missing=0;
run;

Error trying to read authinfo file:C:\Users\james\_authinfo
[Errno 2] No such file or directory: 'C:\\Users\\james\\_authinfo'
Did not find key oda in authinfo file:C:\Users\james\_authinfo

Please enter the IOM user id: james.ssjeong@gmail.com
Please enter the password for IOM user : ········
SAS Connection established. Subprocess id is 13420

7                                                          The SAS System                      Sunday, November  1, 2020 04:57:00 PM

34         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
34       ! ods graphics on / outputfmt=png;
35         
36         /* Import dataset */
37         proc import
38             datafile='/home/u43674369/projects/project01/placement_data.csv'
39             out=df dbms=csv replace;
40         run;
NOTE: Unable to open parameter catalog: SASUSER.PARMS.PARMS.SLIST in update mode. Temporary parameter values will be saved to 
WORK.PARMS.PAR

In [2]:
/* Generating Predictions */
proc format;
	value $job 'Placed' = 1
			   'Not Placed' = 0;
run;

ods select none;

proc logistic data=df;
	class Gender(ref='F') WorkExp(ref='No') / param=ref;
	model Status(event='1') = Degree_P|Gender WorkExp;
	units Degree_P=10;
	format Status $job.;
		store out=joboffer;
run;

ods select all;


/* Input new data */
data newcandid;	
	length Gender $3 WorkExp $3;
	input Degree_P Gender $ WorkExp $;
	datalines;
	51 F Yes
	49 M Yes
	68 F No
	63 M No
	82 F Yes
	76 M Yes
	79 M No
	87 F No
	;
run;


/* Score the new data */
proc plm restore=joboffer;
	score data=newcandid out=scored_candid / ILINK;
	title 'Predictions using PROC PLM';
run;

proc print data=scored_candid;
run;